In [1]:
from xgboost import XGBRegressor
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
df=pd.read_csv("San Diego_featured_data.csv")
df.head()

,All sky irradiance,Clear sky irradiance,ALLSKY ICI,ALLSKY_SFC_PAR_TOT,ALLSKY_SFC_UV_INDEX,CLRSKY_SFC_PAR_TOT
0,3.58,3.68,0.70,63.57,0.55,66.20
1,3.20,3.60,0.62,57.87,0.53,65.50
2,2.52,3.46,0.49,47.44,0.50,64.25
3,2.93,3.48,0.57,55.34,0.49,65.82
4,0.78,3.42,0.15,16.23,0.21,64.96


In [3]:
df.shape

(1917, 6)

In [4]:
df.isnull().sum()

All sky irradiance      0
Clear sky irradiance    0
ALLSKY ICI              0
ALLSKY_SFC_PAR_TOT      0
ALLSKY_SFC_UV_INDEX     0
CLRSKY_SFC_PAR_TOT      0
dtype: int64

In [5]:
df.describe()

,All sky irradiance,Clear sky irradiance,ALLSKY ICI,ALLSKY_SFC_PAR_TOT,ALLSKY_SFC_UV_INDEX,CLRSKY_SFC_PAR_TOT
count,1917.000000,1917.000000,1917.000000,1917.000000,1917.000000,1917.000000
mean,4.983871,6.196766,0.591205,93.750250,-25.176808,114.777783
std,1.650000,1.759774,0.135167,31.056769,161.037349,33.397850
min,0.710000,3.170000,0.100000,14.970000,-999.000000,57.500000
25%,3.710000,4.500000,0.500000,68.450000,0.740000,82.050000
50%,4.950000,6.370000,0.620000,93.840000,1.410000,117.660000
75%,6.240000,7.890000,0.700000,117.910000,1.980000,147.820000
max,8.860000,9.000000,0.780000,162.740000,2.980000,162.850000


In [6]:
y= df[['All sky irradiance']]
scaler_X = StandardScaler()
X_scaled = scaler_X.fit_transform(df)

scaler_y = StandardScaler()
y_scaled = scaler_y.fit_transform(y.values.reshape(-1, 1)).flatten() 

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, test_size = 0.2,shuffle=False)

In [8]:
model=XGBRegressor(tree_method='approx',subsamples=0.7,n_estimators=300,max_depth=9,learning_rate=0.1,colsample_bytree=1)
import time
start=time.time()
model.fit(X_train,y_train)
end=time.time()
continential= end-start
print("convergence time:",continential)

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [11:26:43] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "subsamples" } are not used.

  warnings.warn(smsg, UserWarning)


convergence time: 0.38049888610839844


In [9]:
start=time.time()
pred = model.predict(X_test)
end=time.time()
continential=end-start
print("inference time:",continential)
pred



inference time: 0.002062082290649414


array([-1.5088574 , -0.8819392 , -0.05225241, -0.29454768,  0.8857173 ,
       -1.3465079 ,  0.4512378 , -0.48995304,  0.8843193 ,  0.14350305,
        0.8542748 ,  0.9082435 , -0.00875846,  0.93384427,  1.1665018 ,
        0.84770185,  0.9299567 ,  1.2457522 , -0.7971883 , -0.27442065,
        0.3643816 ,  1.0451764 , -0.21251182,  0.6071014 , -0.79150087,
       -0.89625376,  0.7723434 , -0.49083123, -0.8130795 ,  0.58318704,
       -0.86411893,  0.2054626 ,  1.4085165 ,  1.700697  ,  1.6526868 ,
        0.39847487,  0.9056508 , -0.12465014,  0.11267841,  1.2015045 ,
        1.6808056 ,  1.5756462 ,  1.5222566 ,  1.3153857 ,  1.0074817 ,
        1.0856228 ,  0.77336574, -1.2057664 , -0.41923505,  1.1961718 ,
        1.8431804 ,  1.571384  ,  1.9045733 ,  1.9712667 ,  2.0337055 ,
        1.8218092 ,  0.82951725,  0.142904  , -0.35971397,  1.4443796 ,
        1.790491  ,  1.4893446 ,  1.224348  ,  1.6526668 ,  1.6864315 ,
        0.30736393,  1.019816  ,  2.0490677 ,  2.076817  ,  1.98

In [10]:
pred.shape

(384,)

In [11]:
prediction = pred.reshape(pred.shape[0], 1)

In [12]:
prediction.shape

(384, 1)

In [13]:
predictions = scaler_y.inverse_transform(prediction.reshape(-1, 1))

In [14]:
predictions.shape

(384, 1)

In [15]:
Actual = scaler_y.inverse_transform(y_test.reshape(-1, 1))

In [16]:
Actual.shape

(384, 1)

In [17]:
Calculated =pd.DataFrame(predictions, columns = ['Prediction'])

In [18]:
Actual=pd.DataFrame(Actual, columns=["Actual"])

In [19]:
import os
import pandas as pd

# Concatenate Calculated and Actual DataFrames along the columns
combined = pd.concat([Calculated, Actual], axis=1)

# Define the path to the existing folder
save_dir = "/Users/ubaidahmed/Desktop/Daily_SI_forecasting /San Diego/Models/XgBoost"

# Define the filename within the specified folder
filename = os.path.join(save_dir, "XgBoost_output.csv")

# Save the combined DataFrame to CSV
combined.to_csv(filename, index=False)

print(f"File saved to {filename}")


File saved to /Users/ubaidahmed/Desktop/Daily_SI_forecasting /San Diego/Models/XgBoost/XgBoost_output.csv


# Error Calculation

In [1]:
import pandas as pd
import numpy as np

In [2]:
df=pd.read_csv("XgBoost_output.csv")
features= list(df)
features

['Prediction', 'Actual']

In [3]:
df=df*1000

In [4]:
z= df["Actual"]
x= df["Prediction"]

In [5]:
import math
from sklearn.metrics import mean_squared_error
MSE=mean_squared_error(z,x)
RMSE= math.sqrt(MSE)
from sklearn.metrics import mean_absolute_error
MAE= mean_absolute_error(z,x)
from sklearn.metrics import mean_absolute_percentage_error
MAPE= (mean_absolute_percentage_error(z,x))*100
print("Mean Square Error is:",MSE)
print("Root Mean Square Error is:",RMSE)
print("Mean Absolute Error is:",MAE)
print("Mean Absolute Percentage Error is:",MAPE)


Mean Square Error is: 368.22679518580736
Root Mean Square Error is: 19.18923644092717
Mean Absolute Error is: 10.509029947916659
Mean Absolute Percentage Error is: 0.22635851635022514


In [6]:
max=x.max()
min=x.min()
mean=x.mean()

print("Max",max)
print("Min",min)
print('Mean',mean)

Max 8651.859
Min 1138.9006
Mean 5226.8107575520835


In [7]:
NRMSE= (RMSE/(max-min))*100
print("Normalized Root Mean Square Error:",NRMSE)

Normalized Root Mean Square Error: 0.2554151829314957


In [8]:
NRMSE= (RMSE/(mean))*100
print("Normalized Root Mean Square Error:",NRMSE)

Normalized Root Mean Square Error: 0.36713088211967765


In [9]:
relative_errors = (z - x) / z
squared_relative_errors = relative_errors ** 2
abs_relative_errors = np.abs(relative_errors)

In [10]:
msre = np.mean(squared_relative_errors)
rmsre = np.sqrt(msre)
mare = np.mean(abs_relative_errors)
rmspe = np.sqrt(np.mean((abs_relative_errors * 100) ** 2))

In [11]:
print("Mean Square Relative Error is:",msre)
print("Root Mean Square Relative Error is:",rmsre)
print("Mean Absolute Relative Error is:",mare)
print("Root Mean Square Percentage Error is:",rmspe)

Mean Square Relative Error is: 2.267446662568394e-05
Root Mean Square Relative Error is: 0.004761771374781022
Mean Absolute Relative Error is: 0.0022635851635022515
Root Mean Square Percentage Error is: 0.4761771374781022
